## Description

Given an input string s and a pattern p, implement regular expression matching with support for '.' and '\*' where:

'.' Matches any single character.

"\*" Matches zero or more of the preceding element.

The matching should cover the entire input string (not partial).

## Example 1:

Input: s = "aa", p = "a"

Output: false

Explanation: "a" does not match the entire string "aa".

## Example 2:

Input: s = "aa", p = "a*"

Output: true

Explanation: '\*' means zero or more of the preceding element, 'a'. Therefore, by repeating 'a' once, it becomes "aa".

## Example 3:

Input: s = "ab", p = ".\*"

Output: true

Explanation: ".\*" means "zero or more (\*) of any character (.)".

## Solution

In [2]:
from __future__ import annotations #this was imported so that I could use built in types as generics. 
# Only >3.9 versions of python can use built in types as generics without this import.

In [8]:
# First accepted solution. This one was written with significant assistance, particularly from this video:
# https://www.youtube.com/watch?v=HAA8mgxlov8
# I was able to get the brute force method down but I was unsure how to optimize it for runtime. This video helped me out
# by not only showing me how to restructure my brute force to make it more amenable for caching, but also showing me how to
# combine caching and recursion to reduce runtime. 

# In terms of explaining this problem and its solution, you would probably be better served watching the video linked above rather
# than reading my ramblings. However, here is my very simplified explanation for why this solution works:

# So, to start with, we can see that the problem we have can be reduced down to a binary tree of decisions.
# To see this, we need two pointers that will keep track of where we are in the input string and pattern respectively. We will
# call these 'i' and 'y', where i tracks the input string and y tracks the input pattern. If we begin at i = 0 and y = 0 respectively,
# we can use incrementation and if statements to check (in a binary fashion) whether the sequence of characters we encounter in 
# the input string match the sequence of characters we encounter in the input pattern according to the rules set by the problem 
# description. Now it becomes a question of exactly when we want to increment each pointer respectively, and what kind of if 
# statements we want to use. We will find that asterisks, as defined by the problem, and whether or not we will utilize them to 
# expression match, are pertinent to deciding these.

# We want to use asterisks in places where having multiple of the preceding input_pattern character is necessary to match the input
# string. That is, we're saying that the input_string character at position i matches the input_pattern character at position y.
# Moreover, the next input_pattern character (aka at position y+1) is an asterisk, and the next input_string character at position
# i+1 is the same as the input_string character at position i. For example, aa and a*, where input_string[i = 0] and 
# input_string[i = 1] both equal input_pattern[y = 0], and input_pattern[y = 1] is an asterisk '*'. 
# As you can see, we start out with i and y both being the same character.

# In these cases, we want to increment i by 1 and y by none up until the point at which we no longer need to use * to achieve
# expression matching, that is, up until i becomes a different character than y. At that point, we just increment i by none and 
# y by 2, thereby skipping over the asterisk's position in the input_pattern, and staying at our current position in input_string 
# (where the current position is the one we have reached after having incremented until reaching a character different than its
# preceding one).

# For example, let's use the input string ac and the input pattern a*c.

# i = 0, y = 0
# ac   a*c
# i    y

# string[i] == pattern[y], and pattern[y+1] == '*'
# thus, i +=1

# i = 1, y = 0
# ac   a*c
#  i   y

# string[i] != pattern[y] and pattern[y+1] == '*'
# thus, y +=2

# i = 1, y = 2
# ac   a*c
#  i     y

# string[i] == pattern[y], but pattern[y+1] != '*'/is out of bounds
# thus i +=1 and y+=1

# i = 2, y = 3
# ac   a*c
#   i     y

# string[i] and string[y] both out of bounds, thus expression matching finished and successful, return true.

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        storage = {}
        def depthfirst(i, y):
            # first we check if we have already done the calculation for the current (i, y) pair. If we have, we just return
            # our saved true/false value for that (i, y) pair. Saves us a lot of unnecessary calculation since we're also using
            # recursion.z
            if (i, y) in storage:
                return storage[(i, y)]
            # check if both i and y are out of bounds. To reach this point, all of the characters within the input pattern
            # have already been matched to all of the characters in the input string, indicating that we have successfully matched the 
            # pattern to the string.
            if (i>=len(s) and y>=len(p)):
                return True
            # check if only y is out of bounds. To reach this point, we must have already expression matched all of the input pattern,
            # but there is still some input left within the input string that has not been matched to the pattern. As such, we have not
            # successfully expression matched the entire input string to the entire input pattern, and must return false.
            if y>=len(p):
                return False
            # provided that all of the previous checks have been met, we can check whether the current (i, y) pair match each other.
            # We want to save this particular value because it tells us whether or not we want to utilize an asterisk should we find one
            # in the next position of the input pattern. Check out example above prior to the solution.
            # Remember, '.' matches anything. Assign whether it is true or false to a variable.
            check_current = i<len(s) and (s[i] == p[y] or p[y] == '.')
            # check if the next character in the input pattern is an asterisk (and by extension, check whether there IS a next character
            # within the input pattern)
            if (y+1 < len(p) and p[y+1] == '*'):
                # now we reach the part where we're actually deciding whether or not to increment i and y and by how much. 
                # Given that we have encountered an asterisk in the input pattern, we will either use it or skip it depending on whether
                # or not the input_string character at our current i matches the input pattern character at our current y. We use the same
                # procedure shown in the example provided prior to the solution class.
                storage[(i, y)] = (depthfirst(i, y+2) or # skipping asterisk
                                   (check_current and depthfirst(i+1, y))) # using asterisk
                return storage[(i, y)]
            if check_current:
                # i and y are the same character and y+1 does not have an asterisk, so just increment each by one
                storage[(i, y)] = depthfirst(i+1, y+1)
                return storage[(i, y)]
            #If the expression were to match, we would have already returned true by this point. At this point, return false.
            return False
        return depthfirst(0, 0)
    
# Time complexity of solution is O(n*m), where n and m are the lengths of the input string and input pattern respectively.